In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV, cross_validate, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA
import statsmodels.api as sm
from statsmodels.imputation.mice import MICE, MICEData
# import fancyimpute
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.impute import SimpleImputer
# from imblearn.over_sampling import SMOTE
import time
from collections import Counter
from sklearn.experimental import enable_iterative_imputer 
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer

In [16]:
df = pd.read_csv("/Users/Simon/Documents/UZH/Machine Learning/Clean Mapping/Clean Mapping.csv", sep=';',
                parse_dates=['adate', 'qdate', 'public_date', 'datadate'])

In [10]:
df = df[df['splticrm'] != 'D']
df = df[df['splticrm'] != 'CCC']

In [11]:
#drop all rows where splticrm has NA values:
df = df[df['splticrm'].notna()]

In [12]:
y = pd.factorize(df['splticrm'])[0]
print(pd.factorize(df['splticrm'])[1])

Index(['A', 'A+', 'AA-', 'AAA', 'BBB', 'BBB-', 'AA', 'A-', 'BBB+', 'AA+',
       'CCC+', 'B-', 'B+', 'BB-', 'BB+', 'BB', 'B'],
      dtype='object')


In [13]:
num_cols = ['aftret_invcapx', 'at_turn', 'bm', 'cash_conversion', 'cash_debt', 'cash_lt', 'cash_ratio', 'cfm', 
          'curr_debt', 'curr_ratio', 'de_ratio', 'debt_assets', 'debt_ebitda', 'dpr', 'lt_debt', 'npm', 
          'ocf_lct', 'roa', 'roe', 'short_debt']

# Copy df to df_mice_imputed
df_mice_imputed = df[num_cols].copy(deep=True)

# Initialize IterativeImputer
mice_imputer = IterativeImputer()

# Impute using fit_tranform on diabetes
df_mice_imputed.iloc[:, :] = mice_imputer.fit_transform(df[num_cols])

/Users/Simon/opt/anaconda3/lib/python3.7/site-packages/sklearn/impute/_iterative.py:603: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)


In [14]:
df_mice_imputed.isnull().sum()
df_mice_imputed.head(5)

,aftret_invcapx,at_turn,bm,cash_conversion,cash_debt,cash_lt,cash_ratio,cfm,curr_debt,curr_ratio,de_ratio,debt_assets,debt_ebitda,dpr,lt_debt,npm,ocf_lct,roa,roe,short_debt
0,0.183,0.466,0.252,30.440,0.37,0.675,1.875,0.337,0.36,2.409,0.949,0.485,1.153,0.129,0.486,0.250,0.994,0.232,0.232,0.079
1,0.183,0.466,0.252,30.440,0.37,0.675,1.875,0.337,0.36,2.409,0.949,0.485,1.153,0.129,0.486,0.250,0.994,0.232,0.232,0.079
2,0.183,0.466,0.252,30.440,0.37,0.675,1.875,0.337,0.36,2.409,0.949,0.485,1.153,0.129,0.486,0.250,0.994,0.232,0.232,0.079
3,0.170,0.453,0.233,32.589,0.37,0.679,1.739,0.322,0.39,2.245,0.982,0.493,1.224,0.177,0.458,0.234,0.796,0.227,0.218,0.130
4,0.170,0.453,0.233,32.589,0.37,0.679,1.739,0.322,0.39,2.245,0.982,0.493,1.224,0.177,0.458,0.234,0.796,0.227,0.218,0.130


In [17]:
df_imp = df.drop(['permno', 'CUSIP', 'NCUSIP', 'adate', 'qdate', 'datadate', 'conm', 'tic', 'cusip', 'public_date'], axis=1)
imp = MICEData(data = df_imp, k_pmm=5)
fml = 'splticrm ~ CAPEI + bm + evm + pe_op_basic + pe_op_dil + pe_exi + pe_inc + ps + pcf + dpr + npm + opmbd + opmad + gpm + ptpm + cfm + roa + roe + roce + efftax + aftret_eq + aftret_invcapx + aftret_equity + pretret_noa + pretret_earnat + GProf + equity_invcap + debt_invcap + totdebt_invcap + capital_ratio + int_debt + int_totdebt + cash_lt + invt_act + rect_act + debt_at + debt_ebitda + short_debt + curr_debt + lt_debt + profit_lct + ocf_lct + cash_debt + fcf_ocf + lt_ppent + dltt_be + debt_assets + debt_capital + de_ratio + intcov + intcov_ratio + cash_ratio + quick_ratio + curr_ratio + cash_conversion + inv_turn + at_turn + rect_turn + pay_turn + sale_invcap + sale_equity + sale_nwc + accrual + ptb + PEG_trailing + DIVYIELD + PEG_1yrforward + PEG_ltgforward'
mice = MICE(model_formula=fml, model_class=sm.OLS, data=imp)
impu = mice.fit(5,5)
results = mice.combine(impu)
imputed_data = impu.transform(df_imp.values)
df_imp = pd.DataFrame(imputed_data, columns=df_imp.columns)

print(impu.summary())
print(df_imp.isnull().sum().sum())


ValueError: variable to be imputed has no observed values

In [19]:
df_imput = df.drop(['tic', 'public_date', 'permno', 'CUSIP', 'NCUSIP', 'adate', 'qdate', 'datadate', 'conm', 'tic', 'cusip', 'splticrm'], axis=1)
#df_numeric = df[['Age']].select_dtypes(include=[np.float]).as_matrix()
fml = 'splticrm ~ CAPEI + bm + evm + pe_op_basic + pe_op_dil + pe_exi + pe_inc + ps + pcf + dpr + npm + opmbd + opmad + gpm + ptpm + cfm + roa + roe + roce + efftax + aftret_eq + aftret_invcapx + aftret_equity + pretret_noa + pretret_earnat + GProf + equity_invcap + debt_invcap + totdebt_invcap + capital_ratio + int_debt + int_totdebt + cash_lt + invt_act + rect_act + debt_at + debt_ebitda + short_debt + curr_debt + lt_debt + profit_lct + ocf_lct + cash_debt + fcf_ocf + lt_ppent + dltt_be + debt_assets + debt_capital + de_ratio + intcov + intcov_ratio + cash_ratio + quick_ratio + curr_ratio + cash_conversion + inv_turn + at_turn + rect_turn + pay_turn + sale_invcap + sale_equity + sale_nwc + accrual + ptb + PEG_trailing + DIVYIELD + PEG_1yrforward + PEG_ltgforward'
df_complete=MICE().complete(df_imput)

TypeError: __init__() missing 3 required positional arguments: 'model_formula', 'model_class', and 'data'